In [1]:
import sys

sys.path.append('/Users/marthab/vinted_crawler/vinted_crawler')

import pandas as pd
import numpy as np
import logging

import time
import datetime
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import gspread_dataframe as gd

from confidential.conf_constants import *
from constants.website_constants import *

In [2]:
from confidential.conf_constants import *
from constants.website_constants import *

In [2]:
scope = [SPSH_URL, GAPI_URL]
credentials = ServiceAccountCredentials.from_json_keyfile_name(GKEY_PATH, scope)
gc = gspread.authorize(credentials)
spreadsheet_key = SPSH_KEY
spreadsheet = gc.open_by_key(spreadsheet_key)
wks_name = DATE_SHEET

ws = gc.open_by_key(spreadsheet_key).worksheet(wks_name)

In [14]:
ws = spreadsheet.add_worksheet(title="test", rows="100", cols="20")


In [20]:
ws.update(COLUMN_NAME_CELLS, [COLUMN_NAMES])
ws.format(
        COLUMN_NAME_CELLS,
        {
            "textFormat": {"bold": True},
            "backgroundColor": {"red": 80, "green": 80, "blue": 80},
        })

{'spreadsheetId': '1mE7LuHuDC7Ki0rnFEfx6-DZmLZ4tvu1vBy4-6QXT2kI',
 'replies': [{}]}

In [8]:
today = datetime.datetime.now().date()

In [10]:
ws_date = gc.open_by_key(spreadsheet_key).worksheet(DATE_SHEET)
ws_date.update(DATE_CELL, str(today))

{'spreadsheetId': '1mE7LuHuDC7Ki0rnFEfx6-DZmLZ4tvu1vBy4-6QXT2kI',
 'updatedRange': "'READ ME'!G3",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [13]:
scope = [SPSH_URL, GAPI_URL]
credentials = ServiceAccountCredentials.from_json_keyfile_name(GKEY_PATH, scope)
gc = gspread.authorize(credentials)
spreadsheet_key = SPSH_KEY
spreadsheet = gc.open_by_key(spreadsheet_key)
wks_name = "test"

ws = gc.open_by_key(spreadsheet_key).worksheet(wks_name)
dataframe = pd.DataFrame(ws.get_all_records())



In [14]:
dataframe

,Upload_date,Product_Id,Category,Sub_Category,Brand,Price,Size,Condition,Colour,URL
0,2021-09-30,1343237817-emilio-pucci,jacken,bomberjacken,EMILIO PUCCI,"300,00€",XS / 34 / 6,"NEU, MIT ETIKETT",BUNT,https://www.vinted.de/damen/kleidung/mantel-an...
1,2021-09-30,1342894053-hellblau-weiss-gestreifte-baumwollb...,tops-and-t-shirts,langarmelige-blusen,EMILIO PUCCI,"5,00€",XL / 42 / 14,GUT,"WEISS, HELLBLAU",https://www.vinted.de/damen/kleidung/tops-and-...
2,2021-09-30,1342583176-emilio-pucci-seidenkravattenschal,tucher-and-schals,schals,EMILIO PUCCI,"59,00€",-,SEHR GUT,"SCHWARZ, BUNT",https://www.vinted.de/damen/accessoires/tucher...
3,2021-09-08,1301391866-tasche-jacquemus-le-chiquito,taschen,handtaschen,JACQUEMUS,"330,00€",-,"NEU, MIT ETIKETT",SCHWARZ,https://www.vinted.de/damen/taschen/handtasche...


In [15]:
collected_products = dataframe['URL'].tolist()

In [24]:
collected_products

['https://www.vinted.de/damen/kleidung/mantel-and-jacken/jacken/bomberjacken/1343237817-emilio-pucci',
 'https://www.vinted.de/damen/kleidung/tops-and-t-shirts/langarmelige-blusen/1342894053-hellblau-weiss-gestreifte-baumwollbluse',
 'https://www.vinted.de/damen/accessoires/tucher-and-schals/schals/1342583176-emilio-pucci-seidenkravattenschal',
 'https://www.vinted.de/damen/taschen/handtaschen/1301391866-tasche-jacquemus-le-chiquito']

In [16]:
# setting up chrome options
chrome_options = Options()
# browsing in incognito mode
chrome_options.add_argument("--incognito")
# setting window size to 1920x1080
chrome_options.add_argument("--window-size=1920x1080")

# create instance
driver = webdriver.Chrome(
    chrome_options=chrome_options,
    executable_path="/Users/marthab/Desktop/Python_Labs/vinted_crawler/chromedriver",
)

/Users/marthab/Library/Caches/pypoetry/virtualenvs/vinted-crawler-package-zvLrHxc3-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [34]:
for url in collected_products:
    try:
        driver.get(url)
        filt = dataframe['URL']==url
        dataframe.loc[filt, 'Availability'] = "still online"
    except:
        dataframe.loc[filt, 'Availability'] = "offline since "


In [45]:
today = datetime.datetime.now().date()
for url in collected_products:
    filt = dataframe['URL']==url
    driver.get(url)
    if driver.current_url  == url:
        dataframe.loc[filt, 'Availability'] = f"still online on {today}"
        dataframe.loc[filt, 'Offline since'] = "-"
    else:
        dataframe.loc[filt, 'Availability'] = f"sold"
        dataframe.loc[filt, 'Offline since'] = today


In [46]:
dataframe

,Upload_date,Product_Id,Category,Sub_Category,Brand,Price,Size,Condition,Colour,URL,Availability,Offline since
0,2021-09-30,1343237817-emilio-pucci,jacken,bomberjacken,EMILIO PUCCI,"300,00€",XS / 34 / 6,"NEU, MIT ETIKETT",BUNT,https://www.vinted.de/damen/kleidung/mantel-an...,still online on 2021-10-01,-
1,2021-09-30,1342894053-hellblau-weiss-gestreifte-baumwollb...,tops-and-t-shirts,langarmelige-blusen,EMILIO PUCCI,"5,00€",XL / 42 / 14,GUT,"WEISS, HELLBLAU",https://www.vinted.de/damen/kleidung/tops-and-...,still online on 2021-10-01,-
2,2021-09-30,1342583176-emilio-pucci-seidenkravattenschal,tucher-and-schals,schals,EMILIO PUCCI,"59,00€",-,SEHR GUT,"SCHWARZ, BUNT",https://www.vinted.de/damen/accessoires/tucher...,still online on 2021-10-01,-
3,2021-09-08,1301391866-tasche-jacquemus-le-chiquito,taschen,handtaschen,JACQUEMUS,"330,00€",-,"NEU, MIT ETIKETT",SCHWARZ,https://www.vinted.de/damen/taschen/handtasche...,sold,2021-10-01


In [33]:
# upload new columns to google sheet 

0    test
Name: Availability, dtype: object